Topic: Project 5  
Subject: Creating Final Dataframe   
Date: 12/12/2017  
Name: Zach Heick  

**Summary**: Gathers all data from multiple SQL tables and saves a dataframe for modeling.

In [9]:
import pickle
import os
import boto3
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, select
import os
import pickle
from nltk.corpus import stopwords
from nltk import RegexpTokenizer

In [10]:
engine_name = 'postgresql://' + str(os.environ['zU']) + ':' + str(os.environ['zP']) + '@13.59.54.149:5432/project_kojak'
engine = create_engine(engine_name)

In [11]:
df = pd.read_sql_query('''SELECT 
                           tracks.album_art,
                           tracks.album_name,
                           tracks.duration_ms,
                           tracks.energy,
                           tracks.liveness,
                           tracks.loudness,
                           tracks.speechiness,
                           tracks.tempo,
                           tracks.track_id,
                           tracks.track_name,
                           artists.artist_popularity,
                           artists.artist_name,
                           artists.artist_id,
                           lyrics.lyrics 
                           FROM tracks
                           JOIN artists ON tracks.artist_id = artists.artist_id
                           JOIN lyrics ON tracks.track_id = lyrics.track_id
                           WHERE tracks.energy IS NOT NULL
                           AND lyrics.lyrics IS NOT NULL
                           AND artists.artist_name != 'Kid Rock'
                           ORDER BY tracks.track_name;''', engine)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20273 entries, 0 to 20272
Data columns (total 14 columns):
album_art            20273 non-null object
album_name           20273 non-null object
duration_ms          20273 non-null int64
energy               20273 non-null float64
liveness             20273 non-null float64
loudness             20273 non-null float64
speechiness          20273 non-null float64
tempo                20273 non-null float64
track_id             20273 non-null object
track_name           20273 non-null object
artist_popularity    20273 non-null int64
artist_name          20273 non-null object
artist_id            20273 non-null object
lyrics               20273 non-null object
dtypes: float64(5), int64(2), object(7)
memory usage: 2.2+ MB


In [13]:
import string

In [14]:
df['track_name'] = df['track_name'].apply(lambda i: string.capwords(i))

In [15]:
df.drop_duplicates(subset=['track_name', 'artist_name'], inplace=True)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19987 entries, 0 to 20272
Data columns (total 14 columns):
album_art            19987 non-null object
album_name           19987 non-null object
duration_ms          19987 non-null int64
energy               19987 non-null float64
liveness             19987 non-null float64
loudness             19987 non-null float64
speechiness          19987 non-null float64
tempo                19987 non-null float64
track_id             19987 non-null object
track_name           19987 non-null object
artist_popularity    19987 non-null int64
artist_name          19987 non-null object
artist_id            19987 non-null object
lyrics               19987 non-null object
dtypes: float64(5), int64(2), object(7)
memory usage: 2.3+ MB


In [17]:
df.reset_index(drop=True, inplace=True)

In [18]:
df.drop(df[df['lyrics'].str.contains('<span')].index, inplace=True)

In [19]:
df.reset_index(drop=True, inplace=True)

In [20]:
len(df)

19831

In [21]:
with open('../Data_Files/all_lyrics_dataframe.pickle', 'wb') as f:
    pickle.dump(df, f)

In [22]:
s3 = boto3.client('s3', 
                  aws_access_key_id=os.environ['AWS_ACCESS_KEY'], 
                  aws_secret_access_key=os.environ['AWS_SECRET_KEY'])

In [23]:
s3.upload_file('../Data_Files/all_lyrics_dataframe.pickle', 'metis-project-kojak-bucket', 'all_lyrics_dataframe.pickle')